# 17.K-折交叉验证
划重点：Keras与skleran组合函数，并不能传入callback，这限制了他的使用，自动数据划分也有问题，可以用于一些小型网络，不建议在

In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.datasets import imdb
from keras.preprocessing import sequence
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Embedding, LSTM, TimeDistributed, Input, Bidirectional,Dropout

# 限制显存占比
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 1.0
sess = tf.Session(config=config)
set_session(sess)

# Set random seed
np.random.seed(0)

Using TensorFlow backend.


### 一、加载与处理数据

In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_traina, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_traina), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_traina, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


### 二、模型构建
【sequential 模型】

In [ ]:
def create_network():
    model = Sequential()
    model.add(Embedding(max_features, 128))  # 最多取20000个词，128维输出向量
    # model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))  //原来的
    model.add(LSTM(128,dropout=0.5))
    model.add(Dense(1, activation='sigmoid')) # 输出层
    model.summary()
    # try using different optimizers and different optimizer configs
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

【Functional 模型】

In [3]:
def create_network():
    input = Input(shape=(maxlen,))
    embedded = Embedding(max_features, 128)(input)
    blstm = Bidirectional(LSTM(128), merge_mode='sum')(embedded)
    output = Dense(1, activation='sigmoid')(blstm)
    model = Model(inputs=input, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    model.summary()
    return model

### 三、使用Keras分类器，包含fit

In [4]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=5, 
                                 batch_size=batch_size, 
                                 verbose=1)

### 四、K折交叉验证

In [5]:
cross_val_score(neural_network, x_train, y_train, cv=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 80, 128)           2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,823,297
Trainable params: 2,823,297
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
16666/16666 [==============================] - 108s 6ms/step - loss: 0.4531 - acc: 0.7829
Epoch 2/5
16666/16666 [==============================] - 79s 5ms/step - loss: 0.2303 - acc: 0.9082
Epoch 3/5
16666/16666 [==============================] - 79s 5ms/

array([ 0.81281497,  0.81315253,  0.81099244])